In [1]:
%%bash
curl https://sh.rustup.rs -sSf | sh -s -- -y
source $HOME/.cargo/env



  stable-x86_64-unknown-linux-gnu installed - rustc 1.88.0 (6b00bc388 2025-06-23)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following (note the leading DOT):
. "$HOME/.cargo/env"            # For sh/bash/zsh/ash/dash/pdksh
source "$HOME/.cargo/env.fish"  # For fish
source $"($nu.home-path)/.cargo/env.nu"  # For nushell


info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-06-26, rust version 1.88.0 (6b00bc388 2025-06-23)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
info: installing component 'rust-std'
info: installing component 'rustc'
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'


In [2]:
%%bash
source $HOME/.cargo/env
rm -rf euler_project
cargo new euler_project


    Creating binary (application) `euler_project` package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [3]:
euler_code = """
// Euler method to solve dy/dt = f(t, y)
pub fn euler_method(
    f: impl Fn(f64, f64) -> f64,
    y0: f64,
    t0: f64,
    t_end: f64,
    n: usize,
) -> Vec<(f64, f64)> {
    let h = (t_end - t0) / n as f64;
    let mut result = Vec::new();
    let mut t = t0;
    let mut y = y0;

    result.push((t, y));
    for _ in 0..n {
        y += h * f(t, y);
        t += h;
        result.push((t, y));
    }

    result
}
"""
with open("euler_project/src/euler.rs", "w") as f:
    f.write(euler_code)


In [12]:
main_code = """
mod euler;

use plotters::prelude::*;
use std::fs::File;
use std::io::{BufWriter, Write};

fn ode(t: f64, y: f64) -> f64 {
    t.cos() - y
}

fn y_exact(t: f64) -> f64 {
    0.5 * (t.cos() + t.sin()) + 0.5 * (-t).exp()
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let t0 = 0.0;
    let t_end = 5.0;
    let y0 = 1.0;
    let steps_list = [20, 100, 1000];
    let colors = [&RED, &BLUE, &GREEN];

    let file = File::create("euler_solutions.csv")?;
    let mut writer = BufWriter::new(file);
    writeln!(writer, "n,t,y")?;

    let root = BitMapBackend::new("euler_plot.png", (900, 600)).into_drawing_area();
    root.fill(&WHITE)?;
    let mut chart = ChartBuilder::on(&root)
        .caption("Euler vs Analytical Solution", ("sans-serif", 25))
        .margin(20)
        .x_label_area_size(40)
        .y_label_area_size(50)
        .build_cartesian_2d(t0..t_end, -1.5..1.5)?;
    chart.configure_mesh().draw()?;

    for (i, &n) in steps_list.iter().enumerate() {
        let result = euler::euler_method(ode, y0, t0, t_end, n);
        for (t, y) in &result {
            writeln!(writer, "{},{:.4},{:.6}", n, t, y)?;
        }
        chart.draw_series(LineSeries::new(result.clone(), colors[i]))?
            .label(format!("Euler n = {}", n))
            .legend(move |(x, y)| PathElement::new([(x, y), (x+20, y)], colors[i]));
    }

    let exact: Vec<(f64, f64)> = (0..=1000)
        .map(|i| {
            let t = t0 + i as f64 * (t_end - t0) / 1000.0;
            (t, y_exact(t))
        })
        .collect();

    chart.draw_series(LineSeries::new(exact, &BLACK))?
    .label("Analytical Solution")
    .legend(|(x, y)| PathElement::new([(x, y), (x + 20, y)], &BLACK));

    chart.configure_series_labels().background_style(&WHITE).draw()?;
    println!("✅ Plot and CSV created.");
    Ok(())
}
"""
with open("euler_project/src/main.rs", "w") as f:
    f.write(main_code)


In [15]:
%%bash
# Replace the entire Cargo.toml with a clean one
cat <<EOF > euler_project/Cargo.toml
[package]
name = "euler_project"
version = "0.1.0"
edition = "2021"

[dependencies]
plotters = "0.3"
EOF


In [16]:
%%bash
cd euler_project
source $HOME/.cargo/env
cargo run --release


✅ Plot and CSV created.


   Compiling euler_project v0.1.0 (/content/euler_project)
    Finished `release` profile [optimized] target(s) in 4.18s
     Running `target/release/euler_project`


In [17]:
from google.colab import files
files.download("euler_project/euler_solutions.csv")
files.download("euler_project/euler_plot.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>